In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "http://localhost:9200/"
)

## Load data

In [3]:
import pandas as pd

df = pd.read_csv("myntra_products_catalog.csv").loc[0:499]
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [7]:
df.isna().value_counts()
df.fillna("None",inplace=True)

## Convert to Vector using BERT model

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[-0.039037388, 0.06969534, -0.050332505, 0.066..."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[0.004190086, 0.03711999, 0.04959568, 0.019879..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.03315906, 0.04807079, -0.00513393, -0.0009..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.008998457, 0.06304228, -0.05473925, 0.0280..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[0.0036775572, 0.062430173, -0.049662426, 0.01..."


In [25]:
df["DescriptionVector"] = df["DescriptionVector"].apply(lambda x: x.tolist())

## Create new index in ElasticSearch

In [30]:
es.ping()

True

In [60]:
from indexMapping import indexMapping

es.indices.delete(index="all_products", ignore=[400, 404])
es.indices.create(index="all_products", mappings=indexMapping)


/var/folders/kt/wbxwp98d7dv9qqjxjmg9pzxr0000gn/T/ipykernel_54367/3921231886.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="all_products", ignore=[400, 404])


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

In [61]:
print(es.indices.get_mapping(index="all_products"))

{'all_products': {'mappings': {'properties': {'Description': {'type': 'text'}, 'DescriptionVector': {'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'l2_norm'}, 'Gender': {'type': 'text'}, 'NumImages': {'type': 'long'}, 'Price (INR)': {'type': 'long'}, 'PrimaryColor': {'type': 'text'}, 'ProductBrand': {'type': 'text'}, 'ProductID': {'type': 'long'}, 'ProductName': {'type': 'text'}}}}}


## Ingest data to index

In [62]:
record_list = df.to_dict("records")
print(len(record_list[0]["DescriptionVector"]))

384


In [63]:
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(f"Error with record: {record}")
        print(e)


In [64]:
es.count(index="all_products")

ObjectApiResponse({'count': 500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [67]:
input_keyword = "Blue Shoes"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])
res["hits"]["hits"]


/var/folders/kt/wbxwp98d7dv9qqjxjmg9pzxr0000gn/T/ipykernel_54367/2405159237.py:11: GeneralAvailabilityWarning: This API is in technical preview and may be changed or removed in a future release. Elastic will work to fix any issues, but features in technical preview are not subject to the support SLA of official GA features.
  res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])
/var/folders/kt/wbxwp98d7dv9qqjxjmg9pzxr0000gn/T/ipykernel_54367/2405159237.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])


[{'_index': 'all_products',
  '_id': '10018013',
  '_score': 0.6412538,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}},
 {'_index': 'all_products',
  '_id': '10018075',
  '_score': 0.6412538,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}}]